# Social-Economic Variables Collection and Preprocess

The data collected from EPS data platform.

Based on the following 7 social and economic perspectives, which we assume may have relation with the gender-specific crime, and subject to data accessibility constraints, totally 21 variables are selected from two databases in the EPS platform, Chinses Regional Economic Database and Chinese City Database.



### 1) The general economy (3 Variables)

    GDP per capita (yuan), 

    Secondary industry as percentage to GDP, 

    Service Sector as Percentage to GDP

### 2) Employment and consumption of citizens (5 Variables)

(used for evaluating the quality of residents' lives)

    Number of employees at the end of the year (10,000 Yuan),
    
    Number of the registered unemployed at the end of year, 
    
    Average annual salary per urban employee (Yuan), 
    
    Number of average employees on site (10,000 Yuan), 
    
    Total retail sales of social consumer goods (10,000 Yuan) (5)
    
    
### 3） Education level (3 Variables)
    Government expenditure on Science (10,000 yuan),
    
    Government expenditure on Education (10,000 yuan), 
    
    Number of secondary schools
### 4） Cultural development  (2 Variables)
    Number of theaters, 
    
    Collections of public libraries (100 people)
### 5) Population (4 Variables)
    Population at Year-end (10000 persons), 
    
    Population (registered residence population) at the end of the year (10,000), 
    
    Number of male population (registered residence population) at the end of the year (10,000), 
    
    Number of female population (registered residence population) at the end of the year (10,000)
### 6) Urbanization (2 Variables)
    Natural growth rate (‰), 
    
    Population density (person / square kilometer)
### 7) The quality of government service: 
    Number of employees joining urban basic medical care system,
    
    Persons covered of unemployment insurance

## Read the socio-economic data

The original data are stored in 9 csv files. For each there are at most 3 variables.

For each variable, the data start from 2014 to 2018. 

Read the original data and store each variable in a dataframe.

In [5]:
import pandas as pd

def tran_to_df(file_name):
    '''
    Transfer each csv file which contains 1 or more variables into dataframes, each variable maps a dataframe.
    
    Input:
      a csv file name (string)
    Output:
        a list of dataframes, each dataframe maps a index
    '''
    pd_o = pd.read_csv(file_name,header = None,names = ["city", "2014", "2015", "2016","2017","2018"], encoding = "ansi")
    
    a = list(pd_o[pd_o.city == "北京"].index)
    l_df = []
    
    #if more than one index
    if len(a) > 1:
        for i, item in enumerate(a):
            if i > 0:
                df_i = pd_o.iloc[a[i-1]:item,:]
                l_df.append(df_i)
                if item == a[-1]:
                    df_last = pd_o.iloc[item:,:]
                    l_df.append(df_last)
    else:
        l_df.append(pd_o)
    return l_df


#store all dataframe in a list, each dataframe maps a variable
l_all_df = []
l_len = []
for i in range(1,10):
    l_one = tran_to_df("socio-econ csv/{}-19.csv".format(i))
    l_all_df += l_one

#check if the number of dataframes is equal to the number of variables in original data
len(l_all_df)

21

## Preprocess the data (match and average)

This step includes two goals:
- Match: drop cities in dataframes of socio-economic data but not in crime data.
- Average: for each variable, calculate the mean of 5-year data

In [3]:
#read the crime data
df_crime = pd.read_csv("matches.csv")

#start match
l_match_df = []
l_zxs = ["北京","天津","上海","重庆"]

for df in l_all_df:
    #cities in l_zxs have difference in the name when in crime datafram and when in social data,
    #so first use a for loop to deal with this difference
    for i in l_zxs:
        df.loc[df["city"] == i, "city"] = i + "市"
    
    df = df[df.city.isin(df_crime["location"])]#drop cities those not in the crime data
    df["mean"]=df[["2014", "2015","2016","2017","2018"]].mean(axis=1)#calculate the mean of data per city and store in a new column
    l_match_df.append(df)

#for each index, transform the original dataframe to the dataframe with only 2 columns, 
#one is the name of city, another one is the mean value
l_df_mean = []
for df_i in l_match_df:
    l_df_mean.append(df_i[["city","mean"]])

#merge 21 dataframes into a big datafram    
left = l_df_mean[0]
for i in range(1,21):
    df_final = pd.merge(left, l_df_mean[i], on = "city", how = "outer")
    left = df_final

    
#change the colnames into the English expression of each variable
colnames = ["city", "Population at Year-end (10,000 persons)", 
            "Natural growth rate (‰)",
           "Number of employees at the end of the year (10,000 Persons)", 
            "Number of the registered unemployed at the end of year (10,000 Persons)",
           "Average annual salary per urban employee (Yuan)",
            "Number of average employees on site (10,000 persons)",
           "Population density (person / square kilometer)",
           "Per Capita GDP(yuan)",
           "Secondary industry as percentage to GDP",
           "Service Sector as Percentage to GDP",
           "Total retail sales of social consumer goods (10,000 Yuan)",
           "Government expenditure on Science (10,000 yuan)",
           "Government expenditure on Education (10,000 yuan)",
           "Number of secondary schools",
           "Number of theaters",
            "Number of collections of public library (/100 people)",
            "Number of persons joining urban basic medical care system",
            "Persons covered of unemployment insurance",
           "Population (registered residence population) at the end of the year (10,000)",
            "Number of male population (registered residence population) at the end of the year (10,000)",
            "Number of female population (registered residence population) at the end of the year (10,000)"]
df_final.columns = colnames
df_final# a dataframe with 21 variables'

<ipython-input-3-43db4263eac5>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["mean"]=df[["2014", "2015","2016","2017","2018"]].mean(axis=1)#calculate the mean of data per city and store in a new column


,city,"Population at Year-end (10,000 persons)",Natural growth rate (‰),"Number of employees at the end of the year (10,000 Persons)","Number of the registered unemployed at the end of year (10,000 Persons)",Average annual salary per urban employee (Yuan),"Number of average employees on site (10,000 persons)",Population density (person / square kilometer),Per Capita GDP(yuan),Secondary industry as percentage to GDP,...,"Government expenditure on Science (10,000 yuan)","Government expenditure on Education (10,000 yuan)",Number of secondary schools,Number of theaters,Number of collections of public library (/100 people),Number of persons joining urban basic medical care system,Persons covered of unemployment insurance,"Population (registered residence population) at the end of the year (10,000)","Number of male population (registered residence population) at the end of the year (10,000)","Number of female population (registered residence population) at the end of the year (10,000)"
0,北京市,1355.320,3.542,791.37708,88137.20,124811.882,737.550,816.035,116225.25,19.7350,...,3287752.2,8950368.2,645.6,251.0,430.920,15591149.4,11337030.6,1347.150967,674.3815,672.769467
1,天津市,1043.920,2.530,282.04994,246174.20,91544.412,242.112,857.455,112238.75,44.6875,...,1153322.0,4817743.4,518.6,27.0,161.215,6421408.0,3040200.0,1043.916000,522.1680,521.748000
2,石家庄市,1009.348,8.326,97.60004,52282.25,61379.218,92.150,784.995,53587.40,43.7150,...,101004.0,1560444.2,404.6,20.0,46.160,1747377.8,926655.4,1030.580000,NaN,NaN
3,唐山市,756.232,4.356,85.18016,56397.00,61562.820,75.740,559.740,82182.80,55.7100,...,78359.0,1255766.6,330.4,23.0,30.990,1724581.6,846684.2,756.053333,NaN,NaN
4,邯郸市,1048.640,12.174,71.27770,99918.20,52106.586,64.522,861.785,34702.80,47.4025,...,51359.6,1066941.0,391.2,11.0,16.355,1160446.4,689608.0,1044.733333,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,玉树藏族自治州,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.484680,20.2875,20.197180
256,昌吉回族自治州,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140.124000,70.7700,NaN
257,喀什地区,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,455.712000,229.8680,NaN
258,延边朝鲜族自治州,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.2120,106.586000


In [13]:
#output the final dataframe as a csv file
df_final.to_csv("socioeco_variables_pre.csv")